### Part 1 : Problem Definition, Kaggle Competition selected and why

I chose the following kaggle competition : https://www.kaggle.com/competitions/playground-series-s3e6/data?select=test.csv .
The dataset is based on the Paris Housing Price Prediction dataset (https://www.kaggle.com/datasets/mssmartypants/paris-housing-price-prediction), which is an imaginary dataset of house prices in an urban environment. This dataset was generated from a deep learning model trained on that dataset. 

The features in this dataset are the following : id, squareMeters, numberOfRooms, hasYard, hasPool, floors, cityCode, cityPartRange, numPrevOwners, made, isNewBuilt, hasStormProtector, basement, attic, garage, hasStorageRoom and hasGuestRoom. they are used to preict the target variable price.

I chose a regression task as the majority of datasets I have worked on previously were for classification tasks.

### Part 2 : Data Exploration, Feature Engineering and Data Preparation


First we load the dataset, that has been uploded into databricks(through the create table) and define the schema:

In [0]:
#definition of the schema, we pick the right data types for each column. For the boolean columns, we choose to load them as integers directly to make the creating of models easier (no need to do One Hot Encoding)
schema = 'id INTEGER, squareMeters INTEGER, numberOfRooms INTEGER, hasYard INTEGER, hasPool INTEGER, floors INTEGER, cityCode INTEGER, cityPartRange INTEGER, numPrevOwners INTEGER, made INTEGER, isNewBuilt INTEGER, hasStormProtector INTEGER, basement INTEGER, attic INTEGER, garage INTEGER, hasStorageRoom INTEGER, hasGuestRoom INTEGER, price DOUBLE'

df = spark.read.csv("/FileStore/tables/train.csv", schema=schema, header = True) #we read the file into a spark dataframe
display(df) #and display it

The id column is not meaningful as it just represents the index of the row, so we can drop that column

In [0]:
from pyspark.sql.functions import col

df = df.drop("id") # we drop the id column from the dataframe

There are no null values in the dataset.

#### Data exploration

Now we can move onto the data exploration phase to get a better picture of our dataset and the more import features.  

We can get the descriptive statistics for each feature:

In [0]:
display(df.summary())

Now we can look at the distribution of each feature through visuals. We will plot the histogram and boxplot associated to each distribution. We will use Databrick's graph functionality. First we need to create an SQL table which we name dataset :

In [0]:
df.createOrReplaceTempView("dataset")

We use a simple SQL querry to group the values of squareMeters and count the number of instances associated to each value. We can can see the distribution is very left skewed and there is only one outlier which is very far from the rest of the other values.

In [0]:
%sql
SELECT squareMeters, count(*) AS count
FROM dataset
GROUP BY squareMeters ORDER BY squareMeters

We can isolate the outliers, by taking the values higher than Q3 + 1.5 * IQR, the uperbound used in statistics to find outliers

In [0]:
%sql
SELECT squareMeters, count(*) AS count
FROM dataset WHERE squareMeters > 146181 /* 146181 corresponds to Q3 + 1.5 *IQR */
GROUP BY squareMeters

This value of squareMeters is extremely high and seems unlikely. We can save the outliers in a seperate dataframe to remove them from our dataset 

In [0]:
from pyspark.sql.functions import avg, count

outliers = df.filter(col("squareMeters") > 146181)
display(outliers)

Now let's have a look at numberOfRooms:

In [0]:
%sql
SELECT numberOfRooms, count(*) AS count
FROM dataset
GROUP BY numberOfRooms
ORDER BY numberOfRooms

Here the distribution does not seem approximately normal and there are no outliers.  

Now we can move onto the hasYard and hasPool columns :

In [0]:
%sql
SELECT hasYard, count(*) AS count
FROM dataset
GROUP BY hasYard

In [0]:
%sql
SELECT hasPool, count(*) AS count
FROM dataset
GROUP BY hasPool

Both columns seem to be more or less balanced, with slightly more houses with no yard or no pool.  

Now we can have a look at the floors column:

In [0]:
%sql
SELECT floors, count(*) AS count
FROM dataset
GROUP BY floors
ORDER BY floors

The histogram is left skewed and we can notice a single outlier from looking at the boxplot

In [0]:
%sql
SELECT floors, count(*) AS count
FROM dataset WHERE floors > 100/* 100 corresponds to Q3 + 1.5 * IQR */
GROUP BY floors

In [0]:
display(df.filter(col("floors") > 100))

This value does not seem reasonable at all, so we can also remove this outlier, so we add it to the outliers dataframe 

In [0]:
outliers = outliers.union(df.filter(col("floors") > 100))
display(outliers)

Now let's look at the cityCode column:

In [0]:
%sql
SELECT cityCode, count(*) AS count
FROM dataset
GROUP BY cityCode
ORDER BY cityCode

We can see that the distribution is not a normal distribution and there are a few outliers

In [0]:
display(df.filter(col("cityCode") > 146275))

These are very high values, so we will also remove these outliers.

In [0]:
outliers = outliers.union(df.filter(col("cityCode") > 146275))
display(outliers)

For the cityPartRange column and the numPrevOwners, the distributions also do not seem approximately normal and there are no outliers.

In [0]:
%sql
SELECT cityPartRange, count(*) AS count
FROM dataset
GROUP BY cityPartRange
ORDER BY cityPartRange

In [0]:
%sql
SELECT numPrevOwners, count(*) AS count
FROM dataset
GROUP BY numPrevOwners
ORDER BY numPrevOwners

For the made column the distribution is not normally distributed and there are outliers

In [0]:
%sql
SELECT made, count(*) AS count
FROM dataset
GROUP BY made
ORDER BY made

In [0]:
display(df.filter(col("made") > 2021))


It is not possible for the houses to be made in 10000(this is probaby a data error or anomaly), so we will remove these outliers.

In [0]:
outliers = outliers.union(df.filter(col("made") > 2021))
display(outliers)

The isNewBuilt and hasStormProtector columns are evenly balanced with slightly more houses without these features

In [0]:
%sql
SELECT isNewBuilt, count(*) AS count
FROM dataset
GROUP BY isNewBuilt

In [0]:
%sql
SELECT hasStormProtector, count(*) AS count
FROM dataset
GROUP BY hasStormProtector

Now let's look at the basement column's distribution

In [0]:
%sql
SELECT basement, count(*) AS count
FROM dataset
GROUP BY basement
ORDER BY basement

We can notice that the distribution does not seem normally distributed and there are a few outliers 

In [0]:
display(df.filter(col("basement") > 10000))

These values seem abnormally high, so we will add them to the outliers to remove.

In [0]:
outliers = outliers.union(df.filter(col("basement") > 10000))
display(outliers)

Now moving on to the attic column, we can notice it seems mostly normally distributed and there are a few outliers

In [0]:
%sql
SELECT attic, count(*) AS count
FROM dataset
GROUP BY attic
ORDER BY attic

In [0]:
display(df.filter(col("attic") > 10000))

We choose to remove these outliers as they don't seem like reasonable values

In [0]:
outliers = outliers.union(df.filter(col("attic") > 10000))
display(outliers)

For the garage column, we have an approximately normally distributed feature and there are a few outliers

In [0]:
%sql
SELECT garage, count(*) AS count
FROM dataset
GROUP BY garage
ORDER BY garage

In [0]:
display(df.filter(col("garage") > 1000))

We can remove these outliers as they don't seem consistent with the distribution

In [0]:
outliers = outliers.union(df.filter(col("garage") > 1000))
display(outliers)

The hasStorageRoom column seems almost evenly balanced with slightly more houses without storage rooms

In [0]:
%sql
SELECT hasStorageRoom, count(*) AS count
FROM dataset
GROUP BY hasStorageRoom

For the hasGuestRoom feature, we can see that there are no outliers and the distribution does not seem to be normally distributed

In [0]:
%sql
SELECT hasGuestRoom, count(*) AS count
FROM dataset
GROUP BY hasGuestRoom
ORDER BY hasGuestRoom

Now we can look at the distribution of the target variable price:

In [0]:
%sql
SELECT price, count(*) AS count
FROM dataset
GROUP BY price
ORDER BY price

Similarly the price varible don't seem do be normally distributed and there are no outliers. 

We can conclude that most of our distributions are not normally distributed. This is most likely because this is not real data but rather generated data.  

Now we can explore the correlation between the features and the target variable using a heatmap. We use seaborn to plot the heatmap as the heatmap graph does not work when using the default Databrick graph feature (it only allows to select 2 features and not the entire dataset).

In [0]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df_pandas = df.toPandas()#we transform our spark dataframe to a pandas df

plt.figure(figsize=(10, 6))
sns.heatmap(df_pandas.corr(), cmap='coolwarm') #we use the seaborn heatmap function 
plt.title('Correlation Heatmap of Features')
plt.show()

First we can see that there are no problems of multicolinearity between features, as none of them are correlated between each other. Then we can notice that the target variable is only correlated to the squareMeters feature and no other features, which might be problematic for our models.  

We can explore that correlation better using a scatter plot.

In [0]:
%sql
SELECT price, squaremeters
FROM dataset

As indicated  by the correlation of around 0.6 on the heatmap, we can see on the scatter plot that the squreMeters feature is extremely correlated to the target variable. This will cause overfitting problems for our models. However if we remove this feature we get bad  results.

Now we can remove the outliers from our dataset:

In [0]:
df_without_outliers = df.subtract(outliers)
df_without_outliers.count()

In [0]:
display(df_without_outliers)

#### Feature Selection

As we have a large number of columns we can apply feature selection to reduce the number of features used to train our models.
We will use the UnivariateFeatureSelector (based on the f value) to rank the most meaningful features and take only the 10 most important.



In [0]:
from pyspark.ml.feature import UnivariateFeatureSelector, VectorAssembler

#we take all the features
columns = ["squareMeters", "numberOfRooms", "hasYard", "hasPool", "floors", "cityCode", "cityPartRange", "numPrevOwners", "made", "isNewBuilt", "hasStormProtector", "basement", "attic", "garage", "hasStorageRoom", "hasGuestRoom"]

vector_assembler = VectorAssembler(inputCols=columns, outputCol="features")# we use the vector assembler to put all our features in a single vector
vect_df = vector_assembler.transform(df_without_outliers)
vect_df = vect_df.select("features", col("price").alias("label"))# we add our target varibale price renamed as label

selector = UnivariateFeatureSelector(outputCol="selectedFeatures")
selector.setFeatureType("continuous").setLabelType("continuous").setSelectionThreshold(10) # our features and target are continous varibles and we select a threshold of 10 to take the 10 most important features
model = selector.fit(vect_df)
print("Selected Features:", model.selectedFeatures)

In [0]:
last_column_index = len(df.columns) - 1
selected_features = [0, 1, 4, 8, 13, 11, 5, 10, 6, 15, last_column_index]#we define the indexes of the selected columns and the target variable

df_reduced_features = df_without_outliers.select(*[col(df_without_outliers.columns[i]) for i in selected_features])# we create a new datframe containg only the selcted features and the target and withut the outliers
display(df_reduced_features)

Now we split our data into train and test sets

In [0]:
from pyspark.sql.functions import rand

df_reduced_features = df_reduced_features.withColumnRenamed("price", "label") #we rename the price column to label for our future pipeline
train_df, test_df = df_reduced_features.randomSplit([0.7, 0.3], seed=42)  # we split the dataset 70/30 and set seed to 42 for reproducibility

print(train_df.count(), test_df.count())

In [0]:
display(train_df)

### Part 3: Create models


In terms of preprocessing we will only scale our data(using normalization) as that is an important step for regression tasks and allows to get better models. Linear Regression models are particularly sensitive to unscaled data, while decision trees and ensemble methods are less affected.

To avoid data leakage the test data is scaled on a scaler fitted on the training data only, so that our model does not see the test data distribution during training.  

We define a pipeline containing the vector assembler, the scaler and the model chosen. We are testing out 4 different models : Linear Regression, Decision Tree Regressor, Random Forest Regressor and Gradient Boosted Tree Regressor.  

We do a 5 fold cross validation to get more reliable results instead of simple hold out testing and a grid search to find the best hyperparameters for each model.  

Then we will evaluate each model on multplie different regression metrics : R squared, Root Mean Squared Error, Mean Squared Error, Mean Absolute Error and Explained Varriance

In [0]:
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import LinearRegression

feature_cols = ['squareMeters', 'numberOfRooms', 'floors', 'made', 'garage', 'basement', 'cityCode', 'hasStormProtector', 'cityPartRange','hasGuestRoom', 'label'] # we take only the selected most relevant columns

# We use a VectorAssembler to combine all the features into a single vector
vector_assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# We use a StandardScaler to scale the features to the standard normal distribution (by removing the mean and scaling to unit variance)
scaler = StandardScaler(withMean=True, withStd=True, inputCol="features", outputCol="scaledFeatures")

#We define our Linear Regression Model
lr = LinearRegression(labelCol="label", featuresCol="scaledFeatures")

#We define our parameter grid for hyperparameter tuning for LR taking the most important hyperparameters (explainations of each hyperparameters based on the official documentation)
param_grid_lr = (ParamGridBuilder()
            .addGrid(lr.loss, ["squaredError"]) # already the default parameter: The loss function to be optimized
            .addGrid(lr.regParam, [0.01, 0.1, 1.0]) # Regularization parameter (>= 0), penalizes higher coefficients
            .addGrid(lr.elasticNetParam, [0.0, 0.25, 0.5, 0.75, 1.0]) # The ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty
            .build())

# We define a pipeline that combines all the stages
pipeline_lr = Pipeline(stages=[vector_assembler, scaler, lr])

#We define RegressionEvaluators for each of the regression metrics we will test our models on
evaluator_r2 = RegressionEvaluator(metricName="r2")
evaluator_rmse = RegressionEvaluator(metricName="rmse")
evaluator_mse = RegressionEvaluator(metricName="mse")
evaluator_mae = RegressionEvaluator(metricName="mae")
evaluator_var = RegressionEvaluator(metricName="var")

#we use a 5 fold cross validation to find the optimized model with the best hyperparameters. The best model is chosen based on the RMSE (the model with the lowest RMSE).  We use parallelism to make execution faster and take advantage of spark's distributed nodes
crossval_lr = CrossValidator(estimator=pipeline_lr, evaluator=evaluator_rmse, estimatorParamMaps=param_grid_lr, numFolds=5, parallelism = 4)

# We fit the model on the training data
model_lr = crossval_lr.fit(train_df) 

In [0]:
from pyspark.ml.regression import DecisionTreeRegressor

#We define our Decision Tree Model
dt = DecisionTreeRegressor(labelCol="label", featuresCol="scaledFeatures")

#We define our parameter grid for hyperparameter tuning for DT taking the most important hyperparameters (explainations of each hyperparameters based on the official documentation)
param_grid_dt = (ParamGridBuilder()
            .addGrid(dt.maxDepth, [3, 5, 8, 10]) # Maximum depth of the tree
            .addGrid(dt.minInstancesPerNode, [1, 5, 10]) # Minimum number of instances each child must have after split, if a split causes the left or right child to have fewer than minInstancesPerNode, the split will be discarded as invalid
            .addGrid(dt.minInfoGain, [0.0, 0.1, 0.2]) # Minimum information gain for a split to be considered at a tree node
            .build())

# We define a pipeline that combines all the stages
pipeline_dt = Pipeline(stages=[vector_assembler, scaler, dt])

#we use a 5 fold cross validation to find the optimized model with the best hyperparameters. The best model is chosen based on the RMSE (the model with the lowest RMSE).  We use parallelism to make execution faster and take advantage of spark's distributed nodes
crossval_dt = CrossValidator(estimator=pipeline_dt, evaluator=evaluator_rmse, estimatorParamMaps=param_grid_dt, numFolds=5, parallelism = 4)

# We fit the model on the training data
model_dt = crossval_dt.fit(train_df) 

In [0]:
from pyspark.ml.regression import RandomForestRegressor

#We define our Random Forest Model
rf = RandomForestRegressor(labelCol="label", featuresCol="scaledFeatures")

#We define our parameter grid for hyperparameter tuning for RF taking the most important hyperparameters (explainations of each hyperparameters based on the official documentation)
param_grid_rf = (ParamGridBuilder()
            .addGrid(rf.numTrees, [10, 50, 100]) # Number of trees to train
            .addGrid(rf.maxDepth, [3, 5, 8, 10]) # maxDepth and minInfoGain are the same as for Decision Tree Regressor
            .addGrid(rf.minInfoGain, [0.0, 0.1, 0.2]) 
            .build())

# We define a pipeline that combines all the stages
pipeline_rf = Pipeline(stages=[vector_assembler, scaler, rf])

#we use a 5 fold cross validation to find the optimized model with the best hyperparameters. The best model is chosen based on the RMSE (the model with the lowest RMSE).  We use parallelism to make execution faster and take advantage of spark's distributed nodes
crossval_rf = CrossValidator(estimator=pipeline_rf, evaluator=evaluator_rmse, estimatorParamMaps=param_grid_rf, numFolds=5, parallelism = 4)

# We fit the model on the training data
model_rf = crossval_rf.fit(train_df) 

In [0]:
from pyspark.ml.regression import GBTRegressor

#We define our Gradient-Boosted Tree Model
gbt = GBTRegressor(labelCol="label", featuresCol="scaledFeatures")

#We define our parameter grid for hyperparameter tuning for DBT taking the most important hyperparameters (explainations of each hyperparameters based on the official documentation)
param_grid_gbt = (ParamGridBuilder()
        # we remove maxDepth and minInfoGain as the grid search takes too much time
        #    .addGrid(gbt.maxDepth, [3, 5, 8, 10]) # maxDepth and minInfoGain are similar hyperparameters as for DT and RF
        #    .addGrid(gbt.minInfoGain, [0.0, 0.1, 0.2])
            .addGrid(gbt.maxIter, [10, 20, 50]) # Max number of iterations (>= 0)
            .addGrid(gbt.stepSize, [0.1, 0.05]) # Step size (a.k.a. learning rate) in interval (0, 1] for shrinking the contribution of each estimator
            .build())

# We define a pipeline that combines all the stages
pipeline_gbt = Pipeline(stages=[vector_assembler, scaler, gbt])

#we use a 5 fold cross validation to find the optimized model with the best hyperparameters. The best model is chosen based on the RMSE (the model with the lowest RMSE).  We use parallelism to make execution faster and take advantage of spark's distributed nodes
crossval_gbt = CrossValidator(estimator=pipeline_gbt, evaluator=evaluator_rmse, estimatorParamMaps=param_grid_gbt, numFolds=5, parallelism = 4)

# We fit the model on the training data
model_gbt = crossval_gbt.fit(train_df) 

### Part 4: Model Evaluation and Explanations of Decision

Now we will evaluate and compare our models based on the regression metrics (R squared, Adjusted R squared, Root Mean Squared Error, Mean Squared Error, Mean Absolute Error and Explained Varriance).  

Then we will have a look at the predictions for each model and plot a comparaison between the predictions and the real labels, the residuals plot and the distribution of the residuals.  

For the optimized linear regression model we have the following :

In [0]:
cv_pred_df_lr = model_lr.transform(test_df) #we get the predictions on the test set
 
r2 = evaluator_r2.evaluate(cv_pred_df_lr)# we get R2 and calculate the adjusted R2
n = test_df.count()# n is the total sample size, the number of rows in the test set
p = len(test_df.columns) - 1 #p is the number of independant features, we do len of the columns -1 to remove the target column
r2_adjusted = 1 - (1 - r2) * (n - 1) / (n - p) #formula for adjusted r squared
 
# We print the regression metrics for the Linear Regression model
print(f"Coefficient of Determination (R Squared): {r2}")
print(f"Adjusted R Squared: {r2_adjusted}")
print(f"Root Mean Squared Error: {evaluator_rmse.evaluate(cv_pred_df_lr)}")
print(f"Mean Squared Error: {evaluator_mse.evaluate(cv_pred_df_lr)}")
print(f"Mean Absolute Error: {evaluator_mae.evaluate(cv_pred_df_lr)}")
print(f"Explained Varriance: {evaluator_var.evaluate(cv_pred_df_lr)}")

We seem to have very good results as the R2 is very close to 1, however this also a sign of overfitting

We can look at the predictions made:

In [0]:
display(cv_pred_df_lr.select("label", "prediction"))

We can see that for the first test instances the model is not as accuarte but then the label and prediction values come very close.  

Now we will plot the regression graphs to evaluate our results better.  

We calculate the residuals and add them to the predictions dataset:

In [0]:
from pyspark.sql import SparkSession, functions as F

cv_pred_df_lr = cv_pred_df_lr.withColumn("residuals", F.col("label") - F.col("prediction"))#calculations of the residuals true labels - predicted value

display(cv_pred_df_lr)

We create an SQL table for the LR predictions to make plots using DataBrick's graphing feature

In [0]:
cv_pred_df_lr.createOrReplaceTempView("lr_predictions")

In [0]:
%sql
SELECT label, prediction
FROM lr_predictions

In [0]:
%sql
SELECT residuals, prediction
FROM lr_predictions

In [0]:
%sql
SELECT residuals, count(*) as count
FROM lr_predictions
GROUP BY residuals

As we can see, when using the basic functionalities, we can't properly customize our graphs, so we will use matplotlib and seaborn instead. Matplotlib allows us to ajust the x and y axis to see our visuals better and by default suggests the best view(zoom) to see our data and seaborn offers graph functions to get better looking scatter plots and histograms.

In [0]:
pandas_df_lr = cv_pred_df_lr.select(["label", "prediction"]).toPandas()#we convert our predictions and labels into a pandas dataframe for plotting

x_label = "Actual Value"
y_label = "Predicted Value"

# We use seaborn to create a scatter plot fitted with a regression line
sns.regplot(x="label", y="prediction", data=pandas_df_lr)

plt.xlabel(x_label)
plt.ylabel(y_label)
plt.xlim(0, 10000000) # we adjust the x and y axis to see the linearity better
plt.ylim(0, 10000000)
plt.title("Linear Regression: Actual vs. Predicted Values")
plt.show()


Our predictions and the actual values are very correlated which means we are correctly predicting the values. However this is also a sign of overfitting.

In [0]:
pandas_df_lr["residual"] = pandas_df_lr["label"] - pandas_df_lr["prediction"]# we calculate the residuals

x_label = "Predicted Value"
y_label = "Residuals"

sns.regplot(x="prediction", y="residual", data=pandas_df_lr) #we use the regplot to get the scatter plot, the regression fitted line and confidance interval

plt.xlabel(x_label)
plt.ylabel(y_label)
plt.ylim(-500, 500)#we adjust the y axis to zoom in more
plt.title("Linear Regression : Residuals Plot")
plt.show()

The residuals plot shows that the residuals are focussed around 0 and scattered randomly (with some outliers further away), indicating that the model's errors are unbiased and independent of the predicted values.

In [0]:
sns.distplot(pandas_df_lr["residual"])# seaborn provides a more smoothed version of the histogram

plt.xlabel("Residuals")
plt.ylabel("Density")
plt.xlim(-25, 25)
plt.title("Linear Regression : Distribution of the Residuals")
plt.show()

Here we can see that the residuals distribution is approximately normal, which means that the errors are symmetrical and unbiased.

Now we can have a look at the Decsion tree model:

In [0]:
cv_pred_df_dt = model_dt.transform(test_df) #we get the predictions on the test set

r2 = evaluator_r2.evaluate(cv_pred_df_dt)# we get R2 and calculate the adjusted R2
n = test_df.count()# n is the total sample size, the number of rows in the test set
p = len(test_df.columns) - 1 #p is the number of independant features, we do len of the columns -1 to remove the target column
r2_adjusted = 1 - (1 - r2) * (n - 1) / (n - p)#formula for adjusted r squared
 
# We print the regression metrics for the Decision Tree model
print(f"Coefficient of Determination (R Squared): {r2}")
print(f"Adjusted R Squared): {r2_adjusted}")
print(f"Root Mean Squared Error: {evaluator_rmse.evaluate(cv_pred_df_dt)}")
print(f"Mean Squared Error: {evaluator_mse.evaluate(cv_pred_df_dt)}")
print(f"Mean Absolute Error: {evaluator_mae.evaluate(cv_pred_df_dt)}")
print(f"Explained Varriance: {evaluator_var.evaluate(cv_pred_df_dt)}")

Similarly to the previous model, we get a very high R squared which means the model is likely to be overfitting.  

We can also see that the predictions seem quite close to the true labels, though as for the linear regression model the first few hundred instances are inacurate.

In [0]:
display(cv_pred_df_dt.select("label", "prediction"))

Now we can plot our regression related graphs

In [0]:
pandas_df_dt = cv_pred_df_dt.select(["label", "prediction"]).toPandas() #we convert our predictions and labels into a pandas dataframe for plotting

x_label = "Actual Value"
y_label = "Predicted Value"

# We use seaborn to create a scatter plot fitted with a regression line
sns.regplot(x="label", y="prediction", data=pandas_df_dt)

plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title("Decision Tree : Actual vs. Predicted Values")
plt.show()

The true values and the predicted values seem correlated, but there is more variation(or error) than in the previous model


In [0]:

pandas_df_dt["residual"] = pandas_df_dt["label"] - pandas_df_dt["prediction"] # we calculate the residuals

x_label = "Predicted Value"
y_label = "Residuals"

sns.regplot(x="prediction", y="residual", data=pandas_df_dt)#we use the regplot to get the scatter plot, the regression fitted line and confidance interval

plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title("Decision Tree : Residuals Plot")
plt.show()


The residuals are scattered randomly around 0, but there are bigger error margins/varriation than for the linear regression model

In [0]:
sns.distplot(pandas_df_dt["residual"])# seaborn provides a more smoothed version of the histogram

plt.xlabel("Residuals")
plt.ylabel("Density")
plt.title("Decision Tree: Distribution of the Residuals")
plt.show()

The residuals seem approximately normally distributed so the errors made by the model are at random and unbiased

Now we can move onto the Random Forsest model

In [0]:
cv_pred_df_rf = model_rf.transform(test_df)

r2 = evaluator_r2.evaluate(cv_pred_df_rf)
n = test_df.count()
p = len(test_df.columns) - 1
r2_adjusted = 1 - (1 - r2) * (n - 1) / (n - p)
 
print(f"Coefficient of Determination (R Squared): {r2}")
print(f"Adjusted R Squared: {r2_adjusted}")
print(f"Root Mean Squared Error: {evaluator_rmse.evaluate(cv_pred_df_rf)}")
print(f"Mean Squared Error: {evaluator_mse.evaluate(cv_pred_df_rf)}")
print(f"Mean Absolute Error: {evaluator_mae.evaluate(cv_pred_df_rf)}")
print(f"Explained Varriance: {evaluator_var.evaluate(cv_pred_df_rf)}")
 

We get similar results, this model also seems to be overfitting given the very high R squared.  

When looking at the predictions they seem mostly close to the true values(except for the first few hundred of test examples)

In [0]:
display(cv_pred_df_rf.select("label", "prediction"))

In [0]:
pandas_df_rf = cv_pred_df_rf.select(["label", "prediction"]).toPandas()

x_label = "Actual Value"
y_label = "Predicted Value"

sns.regplot(x="label", y="prediction", data=pandas_df_rf)

plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title("Random Forest : Actual vs. Predicted Values")
plt.show()

The true values and the predicted values are correlated, however there are more outliers than for the previous 2 models

In [0]:
pandas_df_rf["residual"] = pandas_df_rf["label"] - pandas_df_rf["prediction"]

x_label = "Predicted Value"
y_label = "Residuals"

sns.regplot(x="prediction", y="residual", data=pandas_df_rf)

plt.xlabel(x_label)
plt.ylabel(y_label)
plt.ylim(-1000000, 1000000)
plt.title("Random Forest : Residuals Plot")
plt.show()

Looking at the residuals plot, we can see that the residuals are scattered around 0 and we can notice a few points which are a lot further than the rest.

In [0]:
sns.distplot(pandas_df_rf["residual"])

plt.xlabel("Residuals")
plt.ylabel("Density")
plt.xlim(-1000000, 1000000)
plt.title("Random Forest: Distribution of the Residuals")
plt.show()

The residuals seem approximately normally distributed, so the errors made by the model don't show any underlying problems.    


Simialrly we have for the Gradient Boosted Tree:

In [0]:
cv_pred_df_gbt = model_gbt.transform(test_df)

r2 = evaluator_r2.evaluate(cv_pred_df_gbt)
n = test_df.count()
p = len(test_df.columns) - 1
r2_adjusted = 1 - (1 - r2) * (n - 1) / (n - p)
 
print(f"Coefficient of Determination (R Squared): {r2}")
print(f"Adjusted R Squared: {r2_adjusted}")
print(f"Root Mean Squared Error: {evaluator_rmse.evaluate(cv_pred_df_gbt)}")
print(f"Mean Squared Error: {evaluator_mse.evaluate(cv_pred_df_gbt)}")
print(f"Mean Absolute Error: {evaluator_mae.evaluate(cv_pred_df_gbt)}")
print(f"Explained Varriance: {evaluator_var.evaluate(cv_pred_df_gbt)}")


We get similar results to the previous models, with a very high R squared close to 1, meaning that the model is likely to be overfitting.  

As for the other models the predictions seem close except for the first few hundred rows of the test set.

In [0]:
display(cv_pred_df_gbt.select("label", "prediction"))

In [0]:
pandas_df_gbt = cv_pred_df_gbt.select(["label", "prediction"]).toPandas()

x_label = "Actual Value"
y_label = "Predicted Value"

sns.regplot(x="label", y="prediction", data=pandas_df_gbt)

plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title("Gradient Boosted Tree : Actual vs. Predicted Values")
plt.show()

The true values and the predicted values are correlated, we get very similar results to our Decison Tree and Random Forest models.

In [0]:
pandas_df_gbt["residual"] = pandas_df_gbt["label"] - pandas_df_gbt["prediction"]

x_label = "Predicted Value"
y_label = "Residuals"

sns.regplot(x="prediction", y="residual", data=pandas_df_gbt)

plt.xlabel(x_label)
plt.ylabel(y_label)
plt.title("Gradient Boosted Tree : Residuals Plot")
plt.show()

The residual plot shows us that the residuals are scattered around 0, with a lot of variation similar to the Decision Tree model(but slightly better).

In [0]:
sns.distplot(pandas_df_gbt["residual"])

plt.xlabel("Residuals")
plt.ylabel("Density")
plt.title("Gradient Boosted Tree: Distribution of the Residuals")
plt.show()

Similarly to our previous models, the residuals are approximately normally distributed.

When comparing the metrics of our models, we can notice a few things. The Decision Tree, Random Forest, Gradient Boosted Tree
have very similar metrics to each other as they come from the same family of models. All 4 models have very similar R2 and ajusted R2 values as well as explained variance. In term of RMSE, MSE and MAE the Linear Regression model performs worse than the others.   

The results found by ensemble methods like Random Forest are the most reliable as it builds multiple trees and is more likely to capture the variation of the target variable. So we can consider it as our best model.

As mentioned many times, our models seem to be overfitting (the R2 is very close to 1). We have previously noticed that the target variable is very correlated to the squareMeters feature, so we will try to remove that feature and see if we get more reasonable results.

In [0]:
feature_cols2 = ["numberOfRooms", "garage", "floors", "basement", "made", "cityCode", "hasStormProtector", "cityPartRange", "hasGuestRoom"] # we remove squareMeters

#we follow the same steps as previously
vector_assembler2 = VectorAssembler(inputCols=feature_cols2, outputCol="features")

scaler2 = StandardScaler(withMean=True, withStd=True, inputCol="features", outputCol="scaledFeatures")

rf2 = RandomForestRegressor(labelCol="label", featuresCol="scaledFeatures")

pipeline_rf2 = Pipeline(stages=[vector_assembler2, scaler2, rf2])

model_without_squareMeters = pipeline_rf2.fit(train_df)
 
predictions_without_squareMeters = model_without_squareMeters.transform(test_df)

We can see that the predictions are very wrong and completely different than the true value even for rows further down in the test set.

In [0]:
display(predictions_without_squareMeters.select(["label", "prediction"]))

When looking at the evalution metrics, we can see the results are terrible and the model is underfitting. The R2 is very low, only 0.1 which means our model is not able to capture the variance in the dataset and our features are not able to explain the variation of the target variable.

In [0]:
r2 = evaluator_r2.evaluate(predictions_without_squareMeters)
n = test_df.count()
p = len(test_df.columns) - 1 
r2_adjusted = 1 - (1 - r2) * (n - 1) / (n - p)
 
print(f"Coefficient of Determination (R Squared): {r2}")
print(f"Adjusted R Squared: {r2_adjusted}")
print(f"Root Mean Squared Error: {evaluator_rmse.evaluate(predictions_without_squareMeters)}")
print(f"Mean Squared Error: {evaluator_mse.evaluate(predictions_without_squareMeters)}")
print(f"Mean Absolute Error: {evaluator_mae.evaluate(predictions_without_squareMeters)}")
print(f"Explained Varriance: {evaluator_var.evaluate(predictions_without_squareMeters)}")

When trying with the other models, we also get a similar result, so we can conclude that the squareMeters feature is necessary and we cannot exclude it. The squareMeters feature is very correlated to our target variable, most probably beacause of the construction of the dataset as it it synthetic data and not real life data. So it is still better to take our overfitting models, even though they won't be able to properly generalize on unseen real life data.

### Part 5: Discussion of work completed, competition outcomes, and highlighting features of Spark used
This assignment used PySpark to perform a classic machine learning task, predicting house prices. We trained multiple different types of regression models and found the best models for each through a grid search to optimize the hyperparameters. Our best model was the Random Forest Regressor.

For the grid search we could have tunned more different hyperparameters or added more posiible values in our parameter grids, however that would have taken a long time (and not necessarily realistic as we are running it on databricks given that for executions longer than 1 hour the cluster is terminated).

To prevent overfitting we could have tried to performing early stopping, using different regularization techniques for linear regression, or performing pruning on our tree regressors. However, overfitting is most likely due to the nature of the data as we have see that the squareMeters column and the target variable are highly correlated. The other features are not meaningful enough, so we can't build a godd model with the remaining features.

I tried uploading my file for the Kaggle competition, but it did not work as there is no option for a Spark environment. It allowed me to upload my notebook but it look a very long time to process it and the page was unresponive multiple times. In the end it did not even work as seen on the screenshot. 

A few features of Spark were used for this assignment: the distributed processing, the machine learning librarires (MLlib) and the graphing tools provided for SQL querries. The parallelism feature built into Spark allows to run large datasets on multiple nodes to improve efficiency and shorten execution time.